# Play with the model!

In [1]:
import os
import sys

sys.path.append('/Users/efraflores/Desktop/hub/asimov_generator/venv/lib/python3.9/site-packages')
BASE_DIR = '/Users/efraflores/Desktop/EF/Diplo/Asimov' #'/content' if you're using GColab
print(os.listdir(BASE_DIR)) #Don't you forget to upload tokenizer,architecture and weights if you're using GColab

['glove.6B.300d.txt', 'asimov_features.csv', '.DS_Store', 'glove.6B.100d.txt', 'corpus.txt', 'glove.6B.50d.txt', 'rnn_asimov_weights.h5', '3D_WordEmbedding.png', 'project', 'glove.6B.200d.txt', '3D_WordEmbedding.csv', 'doc', 'rnn_asimov_architecture.json', 'glove.6B.zip', 'rnn_asimov_tokenizer.pickle', 'The_Last_Question.pdf']


In [ ]:
!pip install 'tensorflow==2.5' --force-reinstall

     |████████████████████████████████| 195.7 MB 96 kB/s  eta 0:00:011
  Using cached tensorboard-2.5.0-py3-none-any.whl (6.0 MB)
  Using cached termcolor-1.1.0-py3-none-any.whl
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
  Using cached numpy-1.19.5-cp39-cp39-macosx_10_9_x86_64.whl (15.6 MB)
  Using cached gast-0.4.0-py3-none-any.whl (9.8 kB)
     |████████████████████████████████| 462 kB 11.0 MB/s eta 0:00:01
  Using cached keras_nightly-2.5.0.dev2021032900-py2.py3-none-any.whl (1.2 MB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached typing_extensions-3.7.4.3-py3-none-any.whl (22 kB)
  Using cached flatbuffers-1.12-py2.py3-none-any.whl (15 kB)
  Using cached Keras_Preprocessing-1.1.2-py2.py3-none-any.whl (42 kB)
  Using cached absl_py-0.12.0-py3-none-any.whl (129 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached h5py-3.1.0-cp39-cp39-macosx_10_9_x86_64.whl (2.9 MB)
  Using cached grpcio-1.34.1-cp39-cp39-macosx_10_10_x

In [ ]:
!pip install 'numpy==' --force-reinstall

     |████████▏                       | 50.1 MB 6.7 MB/s eta 0:00:222

## Functions

In [7]:
import time
from IPython.display import clear_output

def print_lines(text,word_p_line=10,sleep=False,sleep_time=0.06):
    #Print batches of N words
    lines = ''
    n = 0
    for i,word in enumerate(text.split()):
        lines += '\n' if i//word_p_line==0 else ''
        lines += ' '.join(text.split()[n:n+word_p_line])
        n += word_p_line
    if sleep:
        #Show letter by letter to perform as typing
        for letter,_ in enumerate(lines.strip()):
            clear_output(wait=True)
            print(lines.strip()[:letter+1])
            time.sleep(sleep_time)
    else:
        print(lines.strip())

print_lines('With sleep time: This is how typing a story should looks like! Right?',sleep=True)
print('\nNow without it:')
print_lines('This is just a very basic example of how this function split the text structure in order to have a fixed number of words per line instead of just one line showing the whole text.')

With sleep time: This is how typing a story should
looks like! Right?

Now without it:
This is just a very basic example of how this
function split the text structure in order to have a
fixed number of words per line instead of just one
line showing the whole text.


In [8]:
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences

def Free_AI_Writing(text_model,fitted_tokenizer,next_words=63):
    #Reverse the classes in order to call like this: dict[position] --> word
    dict_classes = {y:x for x,y in fitted_tokenizer.word_index.items()}
    #First string input
    text = str(input(f'Give me the first word or sentence:\n'))
    while len(text.split()) < 1:
        #It does need at least one word at the beginning!
        text = str(input(f'There were no words. Try again\n'))
    else: clear_output()
    whole_text = text
    for _ in range(int(next_words)):
        #Transform the words into the sequence the model needs
        seq = fitted_tokenizer.texts_to_sequences([whole_text])[0]
        seq = pad_sequences([seq],text_model.layers[0].output_shape[1])
        #Predicts an array, brings the most likely class position
        word = np.argmax(text_model.predict(seq),axis=-1)
        #Concat the word with the previous string
        whole_text += ' '+dict_classes[word[0]]
    #Print it as typing
    print_lines(whole_text,sleep=True)

In [9]:
def Input_AI_Writing(text_model,fitted_tokenizer,next_words=10):
    #Reverse the classes in order to call like this: dict[position] --> word
    dict_classes = {y:x for x,y in fitted_tokenizer.word_index.items()}
    #First string input
    text = str(input(f'Give me the first word or sentence:\n'))
    while len(text.split()) < 1:
        #It does need at least one word!
        text = str(input(f"There were no words. Try again\n"))
    else: clear_output()
    whole_text = text
    write = True
    #Until user stop playing with the model predictions
    while write:
        for _ in range(int(next_words)):
            #Transform the words into the sequence the model needs
            seq = fitted_tokenizer.texts_to_sequences([whole_text])[0]
            seq = pad_sequences([seq],text_model.layers[0].output_shape[1])
            #Predicts an array, brings the most likely class position
            word = np.argmax(model.predict(seq),axis=-1)
            #Concat the word with the previous string
            whole_text += ' '+dict_classes[word[0]]
        #Show the resulting text so far
        print_lines(whole_text)
        #Ask user to continue generating text given the updated text
        stop = input('\nContinue generating text? y/n\n')
        while stop not in ('y','n'):
            #It just accepts the correct input
            stop = input('Choose "y" to continue or "n" to exit\n')
        if stop=='n':
            clear_output()
            #Show the final result
            print_lines(whole_text)
            write = False
        else:
            #Ask for the number of the next predicted-words
            next_words = input('How many words more do you want to generate? ')
            need_int = True
            while need_int:
                try:
                    int(next_words)
                except:
                    #It just accepts an integer input
                    next_words = input(f'"{next_words}" is not an integer, try again! ')
                    pass
                else: need_int = False
            else:
                #Ask for another word, sentence or even nothing to continue generating text
                whole_text += ' '+str(input(f'Give me another word/sentence or just press "Enter" to continue:\n'))
                clear_output()

## Load model

In [10]:
import pickle
from keras.models import model_from_json

with open(os.path.join(BASE_DIR,'rnn_asimov_tokenizer.pickle'), 'rb') as f:
    tokenizer = pickle.load(f)

with open(os.path.join(BASE_DIR,'rnn_asimov_architecture.json'), 'r') as f:
    model = model_from_json(f.read())

#Now, we can use this model whenever we want!
model.load_weights(os.path.join(BASE_DIR,'rnn_asimov_weights.h5'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics='accuracy')

NotImplementedError: Cannot convert a symbolic Tensor (bidirectional/forward_gru/strided_slice:0) to a numpy array. This error may indicate that you're trying to pass a Tensor to a NumPy call, which is not supported

## Generate text!

In [ ]:
#Unleash the artificial creativity shows off!
Free_AI_Writing(model,tokenizer)

In [7]:
#Or play with the model writing together <3
Input_AI_Writing(model,tokenizer)

Man's last mind paused before fusion looking over a space
that included nothing but the remaining pieces dregs of one
last dark star and nothing besides
